# Try out gradio

**Load prerequisites**

In [57]:
# Prerequisites
from tabulate import tabulate
from transformers import pipeline
import json
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import time

# Load the model and create a pipeline for zero-shot classification (1min loading + classifying with 89 labels)
classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-base")
model_name = 'model_cross-encoder-nli-deberta-v3-base'
# tried:
# cross-encoder/nli-deberta-v3-large  gave error
# MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli
# facebook/bart-large-mnli
# sileod/deberta-v3-base-tasksource-nli

# get candidate labels
with open("packing_label_structure.json", "r") as file:
    candidate_labels = json.load(file)
keys_list = list(candidate_labels.keys())

# Load test data (in list of dictionaries)
with open("test_data.json", "r") as file:
    packing_data = json.load(file)
# Extract all trip descriptions and trip_types
trip_descriptions = [trip['description'] for trip in packing_data]
trip_types = [trip['trip_types'] for trip in packing_data]

# Access the first trip description
first_trip = trip_descriptions[1]
# Get the packing list for the secondfirst trip
first_trip_type = trip_types[1]

print(f"First trip: {first_trip} \n")
print(f"Trip type: {first_trip_type}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


First trip: We are a couple in our thirties traveling to Vienna for a three-day city trip. We’ll be staying at a friend’s house and plan to explore the city by sightseeing, strolling through the streets, visiting markets, and trying out great restaurants and cafés. We also hope to attend a classical music concert. Our journey to Vienna will be by train. 

Trip type: ['city trip', ['sightseeing'], 'variable weather / spring / autumn', 'luxury (including evening wear)', 'casual', 'indoor', 'no own vehicle', 'no special condition', '3 days']


In [58]:
# function that returns pandas data frame with predictions

cut_off = 0.5  # used to choose which activities are relevant

def pred_trip(trip_descr, trip_type, cut_off):
    # Create an empty DataFrame with specified columns
    df = pd.DataFrame(columns=['superclass', 'pred_class'])
    for i, key in enumerate(keys_list):
        if key == 'activities':
            result = classifier(trip_descr, candidate_labels[key], multi_label=True)
            indices = [i for i, score in enumerate(result['scores']) if score > cut_off]
            classes = [result['labels'][i] for i in indices]
        else:
            result = classifier(trip_descr, candidate_labels[key])
            classes = result["labels"][0]
        print(result)
        print(classes)
        print(i)
        df.loc[i] = [key, classes]
    df['true_class'] = trip_type
    return df

In [59]:
# function for accuracy, perc true classes identified and perc wrong pred classes

def perf_measure(df):
    df['same_value'] = df['pred_class'] == df['true_class']
    correct = sum(df.loc[df.index != 1, 'same_value'])
    total = len(df['same_value'])
    accuracy = correct/total
    pred_class = df.loc[df.index == 1, 'pred_class'].iloc[0]
    true_class = df.loc[df.index == 1, 'true_class'].iloc[0]
    correct = [label for label in pred_class if label in true_class]
    num_correct = len(correct)
    correct_perc = num_correct/len(true_class)
    num_pred = len(pred_class)
    wrong_perc = (num_pred - num_correct)/num_pred
    df_perf = pd.DataFrame({
    'accuracy': [accuracy],
    'true_ident': [correct_perc],
    'false_pred': [wrong_perc]
    })
    return(df_perf)

**Now do the same for all trips**

In [60]:
result_list = []
performance = pd.DataFrame(columns=['accuracy', 'true_ident', 'false_pred'])

start_time = time.time()

for i in range(len(trip_descriptions)):
    current_trip = trip_descriptions[i]
    current_type = trip_types[i]
    df = pred_trip(current_trip, current_type, cut_off = 0.5)
    print(df)
    
    # accuracy, perc true classes identified and perc wrong pred classes
    performance = pd.concat([performance, perf_measure(df)])
    print(performance)
    
    result_list.append(df)

end_time = time.time()

elapsed_time = end_time - start_time

{'sequence': 'I am planning a trip to Greece with my boyfriend, where we will visit two islands. We have booked an apartment on each island for a few days and plan to spend most of our time relaxing. Our main goals are to enjoy the beach, try delicious local food, and possibly go on a hike—if it’s not too hot. We will be relying solely on public transport. We’re in our late 20s and traveling from the Netherlands.', 'labels': ['micro-adventure / weekend trip', 'digital nomad trip', 'beach vacation', 'festival trip', 'city trip', 'cultural exploration', 'road trip (car/camper)', 'camping trip (wild camping)', 'long-distance hike / thru-hike', 'hut trek (winter)', 'ski tour / skitour', 'snowboard / splitboard trip', 'nature escape', 'yoga / wellness retreat', 'hut trek (summer)', 'camping trip (campground)'], 'scores': [0.9722680449485779, 0.007802918087691069, 0.0075571718625724316, 0.0022959215566515923, 0.0021305829286575317, 0.001222927705384791, 0.0009879637509584427, 0.0008052966441

ZeroDivisionError: division by zero

**Compute average performance measures**

In [61]:
# Extract "same_value" column from each DataFrame
sv_columns = [df['same_value'] for df in result_list]  # 'same' needs to be changed
sv_columns.insert(0, result_list[0]['superclass'])

# Combine into a new DataFrame (columns side-by-side)
sv_df = pd.concat(sv_columns, axis=1)

print(sv_df)

IndexError: list index out of range

In [62]:
# Compute accuracy per superclass (row means of same_value matrix excluding the first column)
row_means = sv_df.iloc[:, 1:].mean(axis=1)

df_row_means = pd.DataFrame({
    'superclass': sv_df['superclass'],
    'accuracy': row_means
})

print(df_row_means)

           superclass  accuracy
0       activity_type       0.8
1          activities       0.0
2   climate_or_season       0.5
3    style_or_comfort       0.3
4          dress_code       0.8
5       accommodation       0.8
6      transportation       0.7
7  special_conditions       0.2
8    trip_length_days       0.6


In [ ]:
# Compute performance measures per trip (mean for each column of performance table)
column_means = performance.mean()
print(column_means)

# Plot histograms for all numeric columns
performance.hist(bins=10, figsize=(10, 6))
plt.tight_layout()
plt.show()

In [ ]:
# save results
# Structure to save
model_result = {
    'model': model_name,
    'predictions': result_list,
    'performance': performance,
    'perf_summary': column_means,
    'perf_superclass': df_row_means,
    'elapsed_time': elapsed_time
}

# File path with folder
filename = os.path.join('results', f'{model_name}_results.pkl')

# Save the object
with open(filename, 'wb') as f:
    pickle.dump(model_result, f)

In [ ]:
print(elapsed_time/60)

**Load and compare results**

In [54]:
# Folder where your .pkl files are saved
results_dir = 'results'

# Dictionary to store all loaded results
all_results = {}

# Loop through all .pkl files in the folder
for filename in os.listdir(results_dir):
    if filename.endswith('.pkl'):
        model_name = filename.replace('_results.pkl', '')  # Extract model name
        file_path = os.path.join(results_dir, filename)
        
        # Load the result
        with open(file_path, 'rb') as f:
            result = pickle.load(f)
            all_results[model_name] = result

# Compare performance across models
for model, data in all_results.items():
    print(f"Model: {model}")
    print(f"Performance Summary:\n{data['perf_summary']}")
    print("-" * 40)


# Compare performance across models
for model, data in all_results.items():
    print(f"Model: {model}")
    print(f"Performance Summary:\n{data['perf_superclass']}")
    print("-" * 40)

Model: model_MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli
Performance Summary:
accuracy      0.522222
true_ident    0.841667
false_pred    0.572381
dtype: float64
----------------------------------------
Model: model_a_facebook-bart-large-mnli
Performance Summary:
accuracy      0.454545
true_ident    0.689394
false_pred    0.409091
dtype: float64
----------------------------------------
Model: model_b_sileod-deberta-v3-base-tasksource-nli
Performance Summary:
accuracy      0.500000
true_ident    0.666667
false_pred    0.551667
dtype: float64
----------------------------------------
Model: model_MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli
Performance Summary:
           superclass  accuracy
0       activity_type       0.8
1          activities       0.0
2   climate_or_season       0.5
3    style_or_comfort       0.3
4          dress_code       0.8
5       accommodation       0.8
6      transportation       0.7
7  special_conditions       0.2
8    trip_length_days       0.6
------------

In [69]:
# Compare across models
for model, data in all_results.items():
    print(f"Model: {model}")
    print(f"Time in minutes for 10 trips:\n{data['elapsed_time']/60}")
    print("-" * 40)

Model: model_MoritzLaurer-DeBERTa-v3-base-mnli-fever-anli
Time in minutes for 10 trips:
83.45150986512502
----------------------------------------
Model: model_a_facebook-bart-large-mnli


KeyError: 'elapsed_time'

**Use gradio for user input**

In [66]:
# use model with gradio
from transformers import pipeline
import gradio as gr

# make a function for what I am doing
def classify(text):
    df = pd.DataFrame(columns=['Superclass', 'class'])
    for i, key in enumerate(keys_list):
        # Run the classification (ca 30 seconds classifying)
        if key == 'activities':
            result = classifier(text, candidate_labels[key], multi_label=True)
            classes = [result['labels'][i] for i in indices]
        else:
            result = classifier(text, candidate_labels[key])
            classes = result["labels"][0]
        print(i)
        df.loc[i] = [key, classes]

    return df

demo = gr.Interface(
    fn=classify,
    inputs="text",
    outputs="dataframe",
    title="Zero-Shot Classification",
    description="Enter a text describing your trip",
)

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://aa06d5d85ffadaa92b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8


Use model with gradio

In [4]:
# Define the Gradio interface
def classify(text):
    return classifier(text, class_labels)

demo = gr.Interface(
    fn=classify,
    inputs="text",
    outputs="json",
    title="Zero-Shot Classification",
    description="Enter a text describing your trip",
)

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://0f70ba5369d721cf8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
